This is a notebook to test the conversion of :

**PyTorch -.pth -->  ONNX -->  TensorflowRep object --> protobuf -.pb**


In [8]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!pip install tensorflow torch onnx onnx-tf

# you can just do 
#!pip install --upgrade --force-reinstall torch


In [0]:
#torch_url=f"https://download.pytorch.org/whl/nightly/cpu/torch-1.2.0.dev20190804%2Bcpu-cp36-cp36m-linux_x86_64.whl"
#!pip install -U {torch_url} torchvision

!pip install torchvision

### PyTorch to ONNX

In [0]:

import torch.onnx
import torchvision


#help(torch.onnx.export)

In [0]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
checkpoint_name = 'akash-mobilenet_v2-FER1-60perc.pt'
onnx_export_path = base_path + "ONNX/mobilenet_v2-bc2.onnx"


In [0]:

# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.

# A model class instance (class not shown)
model_pytorch = torchvision.models.mobilenet_v2(pretrained=False)
model_pytorch.classifier[1] = torch.nn.Linear(1280, 7)

#print(model)

# Load the weights from a file (.pth usually)
state_dict = torch.load(base_path + checkpoint_name, map_location=torch.device('cpu'))

# Load the weights now into a model net architecture defined by our class
model_pytorch.load_state_dict(state_dict)

# Single pass of dummy variable required
# Create the right input shape (e.g. for an image)
dummy_input = torch.randn(1, 3, 256, 256)
dummy_output = model_pytorch(dummy_input)
dummy_output = torch.softmax(dummy_output, 1)
print(dummy_output)

# important: explicitly name the input and output nodes! Useful for later 
torch.onnx.export(model_pytorch, 
                  dummy_input, 
                  onnx_export_path,
                  input_names=['input'], 
                  output_names=['output'])


tensor([[0.1051, 0.0037, 0.1743, 0.0883, 0.3438, 0.0218, 0.2629]],
       grad_fn=<SoftmaxBackward>)


### Import ONNX to Tensorflow

In [0]:
!pip install onnx
!pip install git+https://github.com/onnx/onnx-tensorflow.git


In [0]:
#!pip uninstall onnx_tf
!pip install onnx_tf

In [0]:
import onnx
import warnings

# If we use a TF version lower than 1.13, we will get the AttributeError:
# "AttributeError: module 'tensorflow' has no attribute 'math'" or similar
from onnx_tf.backend import prepare

warnings.filterwarnings('ignore') # Ignore all the warning messages in this tutorial

onnx_model = onnx.load(onnx_export_path) # Load the ONNX file

tf_rep = prepare(onnx_model) # Import the ONNX model to Tensorflow



If you have specified the input and output names in the torch.onnx.export function, you should see the keys ‘input’ and ‘output’ along with their corresponding values, as shown in the snippet below. The names **‘input:0’** and **'add_10:0'** will be used during inference in TensorFlow.

In [0]:
# Now we have tf_rep, which is a python class containing four members: 
# graph, inputs, outputs, and tensor_dict.

print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

['input']
-----
['output']
-----
{'classifier.1.bias': <tf.Tensor 'Const:0' shape=(7,) dtype=float32>, 'classifier.1.weight': <tf.Tensor 'Const_1:0' shape=(7, 1280) dtype=float32>, 'features.0.0.weight': <tf.Tensor 'Const_2:0' shape=(32, 3, 3, 3) dtype=float32>, 'features.0.1.bias': <tf.Tensor 'Const_3:0' shape=(32,) dtype=float32>, 'features.0.1.num_batches_tracked': <tf.Tensor 'Const_4:0' shape=() dtype=int64>, 'features.0.1.running_mean': <tf.Tensor 'Const_5:0' shape=(32,) dtype=float32>, 'features.0.1.running_var': <tf.Tensor 'Const_6:0' shape=(32,) dtype=float32>, 'features.0.1.weight': <tf.Tensor 'Const_7:0' shape=(32,) dtype=float32>, 'features.1.conv.0.0.weight': <tf.Tensor 'Const_8:0' shape=(32, 1, 3, 3) dtype=float32>, 'features.1.conv.0.1.bias': <tf.Tensor 'Const_9:0' shape=(32,) dtype=float32>, 'features.1.conv.0.1.num_batches_tracked': <tf.Tensor 'Const_10:0' shape=() dtype=int64>, 'features.1.conv.0.1.running_mean': <tf.Tensor 'Const_11:0' shape=(32,) dtype=float32>, 'fea

In [0]:
!pip uninstall tensorflow-gpu
!pip install --upgrade tensorflow

#!pip install tensorflow==1.7.1

# install this TF v1.7.1 to avoid incompatibility issues in Unity with 
# TensorflowSharp v0.5
# However, if we use a TF version lower than 1.13, 
# we will get the AttributeError:
#"AttributeError: module 'tensorflow.math' has no attribute" or similar

import tensorflow as tf

print(tf.__version__)


In [0]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

### Prepare a dummy image for inference

In [0]:

import numpy as np
from IPython.display import display
from PIL import Image

path_to_img = base_path + 'fer2013/PrivateTest/2/PrivateTest_1162474.jpg'
#path_to_img = base_path + 'fer2013/testface-happy.jpg'

img = Image.open(path_to_img).resize((256, 256))

new_img = np.broadcast_to(img, (1, 3, 256, 256)).copy()
print(new_img.shape)

display(new_img) # show the image

img_ycbcr = img.convert("YCbCr")
img_y, img_cb, img_cr = img_ycbcr.split()


#test_face = np.asarray(img_y, dtype=np.float32)[np.newaxis, np.newaxis, :, :]

(1, 3, 256, 256)


array([[[[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]],

        [[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]],

        [[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]]]], dtype=uint8)

In [0]:
# run the network
#dummy_face = torch.randn(1, 3, 224, 224)

print(new_img.shape)

test_face_run = tf_rep.run(new_img)._0

print(test_face_run)

(1, 3, 256, 256)
[[ 0.2374446  -3.5035207   0.17487492 -0.596612   -0.21957415  0.8263142
  -0.06455269]]


### Tensorflow -.h5 or ONNX to -.pb

In [0]:
#from tensorflow.python.saved_model import builder as pb_builder
import tensorflow as tf

base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 

export_path_pb = base_path + "ONNX/akash-bc2.pb"


In [0]:
# load the model and save it as pb. 
# If the model is not in h5 format, convert it to h5 first.
#pre_model = tf.keras.models.load_model('path/to/your/model.model')
#pre_model.save('path/to/your/model.h5')


# export the TensorflowRep object to a -.pb file which can be imported to unitiy.
tf_rep.export_graph(export_path_pb) 

### Inspect the -.pb file. Print out some node names

In [9]:
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/' 
pb_path = base_path + 'ONNX/akash-bc2.pb'
#pb_path = base_path + 'ONNX/weights_best.pb'

gf = tf.GraphDef()
gf.ParseFromString(open(pb_path,'rb').read())

10783695

In [0]:
[n.name + '=>' +  n.op for n in gf.node if n.op in ( 'Softmax','Mul', 'Placeholder', 'Reshape', 'Predictions', 'Add')]


### Doing inference in TensorFlow

see if the resultant TensorFlow model can do inference as intended. Loading a TensorFlow model from a .pb file can be done by defining the following function.

In [0]:
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [0]:
tf_graph = load_pb(export_path_pb)
sess = tf.Session(graph=tf_graph)

input_tensor = tf_graph.get_tensor_by_name('input:0')
output_tensor = tf_graph.get_tensor_by_name('add_10:0')

output = sess.run(output_tensor, feed_dict={input_tensor: dummy_input})
print(output)

[[ 0.43117827 -4.547673   -0.07455972 -0.43664512  1.002899   -0.88042116
   0.58427036]]


# **From -.pb to -.tflite**

In [10]:

pb_file = export_path_pb
tflite_file = 'my_tflite.lite'
pb_file

'./gdrive/My Drive/Colab Notebooks/Fer-dataset/ONNX/akash-bc2.pb'

In [13]:

converter = tf.lite.TFLiteConverter.from_frozen_graph(
    pb_file, 
    ['input'], 
    ['Add']
)

tflite_model = converter.convert()

open(tflite_file,'wb').write(tflite_model)



ConverterError: ignored